In [1]:
!pwd

/content


In [2]:
import os
from google.colab import drive

drive.mount('./mount')#运行后点击链接，复制密钥
 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/mount/My Drive/Colab Notebooks/kaggle' 

os.chdir('/content/mount/My Drive/Colab Notebooks/kaggle') #切换到kaggle文件夹
!kaggle datasets download -d amananandrai/ag-news-classification-dataset

Drive already mounted at ./mount; to attempt to forcibly remount, call drive.mount("./mount", force_remount=True).
ag-news-classification-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip -A ag-news-classification-dataset.zip

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [4]:
import pandas as pd
import os

cwd=os.getcwd()
print(cwd)

data = pd.read_csv(cwd+'/train.csv')
data = pd.DataFrame(data=data)
data.columns = data.columns.str.replace(" ","_")
data.columns = data.columns.str.lower()
data['class_name']=data['class_index'].map({1:"World",2:"Sports",3:"Business",4:"Sci_Tech"})

/content/mount/My Drive/Colab Notebooks/kaggle


In [5]:
data[:10]

,class_index,title,description,class_name
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business
5,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...,Business
6,3,Money Funds Fell in Latest Week (AP),AP - Assets of the nation's retail money marke...,Business
7,3,Fed minutes show dissent over inflation (USATO...,USATODAY.com - Retail sales bounced back a bit...,Business
8,3,Safety Net (Forbes.com),Forbes.com - After earning a PH.D. in Sociolog...,Business
9,3,Wall St. Bears Claw Back Into the Black,"NEW YORK (Reuters) - Short-sellers, Wall Stre...",Business


In [6]:
data.class_name.value_counts()

Business    30000
Sci_Tech    30000
Sports      30000
World       30000
Name: class_name, dtype: int64

In [7]:
# 打印前10篇新闻
for i in range(10):
  print("Tittle of Article",i)
  print(data.loc[i,"title"])
  print("\n")

Tittle of Article 0
Wall St. Bears Claw Back Into the Black (Reuters)


Tittle of Article 1
Carlyle Looks Toward Commercial Aerospace (Reuters)


Tittle of Article 2
Oil and Economy Cloud Stocks' Outlook (Reuters)


Tittle of Article 3
Iraq Halts Oil Exports from Main Southern Pipeline (Reuters)


Tittle of Article 4
Oil prices soar to all-time record, posing new menace to US economy (AFP)


Tittle of Article 5
Stocks End Up, But Near Year Lows (Reuters)


Tittle of Article 6
Money Funds Fell in Latest Week (AP)


Tittle of Article 7
Fed minutes show dissent over inflation (USATODAY.com)


Tittle of Article 8
Safety Net (Forbes.com)


Tittle of Article 9
Wall St. Bears Claw Back Into the Black




In [8]:
# 打印前10篇新闻
for i in range(10):
  print("Description of Article",i)
  print(data.loc[i,"description"])
  print("\n")

Description of Article 0
Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


Description of Article 1
Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.


Description of Article 2
Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.


Description of Article 3
Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.


Description of Article 4
AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.


Description o

In [9]:
# 去除噪声
cols=["title","description"]
data[cols]=data[cols].applymap(lambda x : x.replace("\\"," "))
data[cols]=data[cols].applymap(lambda x : x.replace("#36;","$"))
data[cols]=data[cols].applymap(lambda x: x.replace("  "," "))
data[cols]=data[cols].applymap(lambda x: x.strip())

# 写入准备文件
data.to_csv(cwd+'/train_prepared.csv',index=False)

## NLP任务1：命名实体识别

In [10]:
# !pip install -U spacy[cuda,transformers,lookups]
# # !pip install -U spacy-lookups-data
# # !pip install cupy-cuda
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda-autodetect]'
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.4.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 

In [11]:
import spacy
spacy.require_gpu()
print(spacy.require_gpu())
nlp = spacy.load("en_core_web_trf")

True


In [12]:
import pprint

In [13]:
pp=pprint.PrettyPrinter(indent=4)
pp.pprint(nlp.meta)

{   'author': 'Explosion',
    'components': [   'transformer',
                      'tagger',
                      'parser',
                      'attribute_ruler',
                      'lemmatizer',
                      'ner'],
    'description': 'English transformer pipeline (roberta-base). Components: '
                   'transformer, tagger, parser, ner, attribute_ruler, '
                   'lemmatizer.',
    'disabled': [],
    'email': 'contact@explosion.ai',
    'labels': {   'attribute_ruler': [],
                  'lemmatizer': [],
                  'ner': [   'CARDINAL',
                             'DATE',
                             'EVENT',
                             'FAC',
                             'GPE',
                             'LANGUAGE',
                             'LAW',
                             'LOC',
                             'MONEY',
                             'NORP',
                             'ORDINAL',
                             

In [14]:
for i in range(9):
  print("Artile",i)
  print(data.loc[i,"description"])
  print("Text Start End Label")
  doc = nlp(data.loc[i,"description"])
  for token in doc.ents:
    print(token.text,token.start_char,token.end_char,token.label_)
  print("\n")

Artile 0
Reuters - Short-sellers, Wall Street's dwindling band of ultra-cynics, are seeing green again.
Text Start End Label
Reuters 0 7 ORG


Artile 1
Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.
Text Start End Label


CUDARuntimeError: ignored

In [ ]:
for i in range(10):
  print("Description of Article",i)
  print(data.loc[i,"description"])
  print("\n")

In [16]:
import torch
torch.version.cuda

'11.6'